In [1]:
from Crypto.Util import number
import hashlib
import math 

### Helpers

In [2]:
def largest_square_less_than(delta):
    return int(math.floor(math.sqrt(float(delta))))


def four_squares(delta):
    d = int(delta)
    u1 = largest_square_less_than(d)
    u2 = largest_square_less_than(d - math.pow(u1, 2))
    u3 = largest_square_less_than(d - math.pow(u1, 2) - math.pow(u2, 2))
    u4 = largest_square_less_than(d - math.pow(u1, 2) - math.pow(u2, 2) - math.pow(u3, 2))

    uPow =  int(math.pow(u1, 2) + math.pow(u2, 2) + math.pow(u3, 2) + math.pow(u4, 2))
    if uPow == d:
        roots = [u1, u2, u3, u4]
    else:
        roots = [-1, -1, -1, -1]

    return roots

#  The fundamental structure of anonymous credential systems
$$
Identity Provider / Issuer \rightarrow \text{\{Issuance of an anonymous credential\}} \rightarrow User / Prover \rightarrow \text{\{attributes proof\}} \rightarrow  Relying Party / Verifier
$$

# Flow diagram representing the credential issuance protocol 
# (User $\rightarrow$ Issuer)

| Step | User | Public | Issuer |
| --- | --- | --- | --- |
| 0. |  | $\xrightarrow[\hspace{4cm}]{\text{Credential request}}$ |  |
| 1. |  | $\xleftarrow[\hspace{4cm}]{pk_{I} = (n, S, Z, R_1,R_2,\ldots,R_i)}$ | Generate public key $(pk_{I})$ and private key $(sk_I)$ <br/> $sk_I = (p, q)$ |
| 2. | Generate master secret $m_1$ <br/> Random $v'$ <br/> $U = S^{v'}R_1^{m_1}\pmod{n}$ | $\xrightarrow[\hspace{4cm}]{U}$ | |
| 3. |  | $\xleftarrow[\hspace{4cm}]{(A,e,v'')}$ | Generate pre-signature $(A,e,v'')$ <br/><br/> Random $v''$ and prime $e$ <br/><br/> $Q = \frac{Z}{U S^{v''}(R_2^{m_2}R_3^{m_3}\cdots  R_i^{m_i})}\pmod{n}$ <br/><br/> $A = Q^{e^{-1}\pmod{p'q'}}\pmod{n}$|
| 4. | Generate signature $(A,e,v)$ <br/><br/> $v = v'+v''$ <br/><br/> $\hat{Z} = A^e S^v \Bigg(\prod_{i \in A}{R_i^{m_i}}\Bigg)$ <br/><br/> Verify $\hat{Z} == Z$ <br/><br/> User store credential $(\{m_i\}, A, e, v)$  |  | |


## User send to Issuer credential request
### Issuer setup

In [3]:
# For demo purposes to make print more readable and consistent
bitsize = 32
p = 3397833353
q = 2516557159
# p = number.getPrime(bitsize)
# q = number.getPrime(bitsize)

# The public parts
n = p*q
nsize = 2*bitsize
# Generate quadratic residues mod n
R_1 = pow(number.getRandomRange(2, n), 2, n)
R_2 = pow(number.getRandomRange(2, n), 2, n)
R_3 = pow(number.getRandomRange(2, n), 2, n)
R_4 = pow(number.getRandomRange(2, n), 2, n)
R_5 = pow(number.getRandomRange(2, n), 2, n)
R_6 = pow(number.getRandomRange(2, n), 2, n)
S = pow(number.getRandomRange(2, n), 2, n)
Z = pow(number.getRandomRange(2, n), 2, n)

The issuer's public key is $(n, S, Z, R_1,R_2,\ldots,R_i)$ and the private key is $(p, q)$.

## User loads public key and send $U$ to Issuer

In [4]:
# master secret
msize = nsize
m_1 = number.getRandomRange(2, n)

# Generate random integear
v_prim = number.getRandomNBitInteger(msize*nsize + 1)

# Calculate U
U = pow(S, v_prim, n) * pow(R_1, m_1, n)

User sends $(U)$ to the Issuer.

Note that the issuer is unable to calculate the value of the commitment $U$ as he does not have the secret key $m_1$;

## Issuer generate CL signature

In [5]:
# messages
m_2 = number.getRandomRange(2, n)
m_3 = number.getRandomRange(2, n)
m_4 = number.getRandomRange(2, n)
m_5 = number.getRandomRange(2, n)
m_6 = number.getRandomRange(2, n)

# Generate random prime
e = number.getPrime(nsize + 1)
e_inv = pow(e, -1, (p-1)*(q-1))
# v is selected as integer
v_pprim = number.getRandomNBitInteger(msize*nsize + 1)

# Calculate dot
R_vector = pow(R_2, m_2, n) * pow(R_3, m_3, n) * pow(R_4, m_4, n) * pow(R_5, m_5, n) * pow(R_6, m_6, n)

# Calculate commitment
commitment_vector = U * pow(S, v_pprim, n) * R_vector
commitment_vector_inv = pow(commitment_vector, -1, n)

# Calculate Q
Q = (Z * commitment_vector_inv) % n
# Calculate signature part A
A = pow(Q, e_inv, n)

Issuer sends pre-signature $(A,e,v'')$ to the User

## User complete signature $(A, e, v)$

In [6]:
# compute v
v = v_prim + v_pprim

## User verify the signature $(A,e,v)$

In [ ]:
# verify
R_dot = pow(R_1, m_1, n) * R_vector
Z_hat = (pow(A, e, n) * pow(S, v, n) * R_dot) % n
assert(Z_hat == Z)

User store credential $(A, e, v,\{m_i\}_{i \in A})$

# Flow diagram representing the verification protocol 
# (User $\rightarrow$ Verifier)

Let A be the set of all attribute identifiers present in the credential schema
such that $A_r$ contains revealed attributes to the Verifier and $A_h$ contains unrevealed (hidden) attributes which are kept secret.

$H$ is a cryptographically secure hash function used to compute the challange $D$ which is a hash value of the commitments and common values.


#### Flow diagram without predicate

| Step | Verifier | Public <br/> $n, S, Z, \{{R_i}\}_{i \in A},\{m_i\}_{i \in A_r}$ <br/> $\text{System parameter: } \beta$ <br/> $\text{Secure hash function: } H$ | User <br/> $\text{Secret: } A, e, v,\{m_i\}_{i \in A_h}$ |
| --- | --- | --- | --- |
| 0. | Random $n_1$ | $\xrightarrow[\hspace{4cm}]{\text{Proof request and } n_1}$ |  |
| 1. |  | $\xleftarrow[\hspace{4cm}]{Pr_C=(A',\hat{e},\hat{v},\{\hat{m_j}\}_{j \in A_h})}$ | Random $r,\bar{e},\bar{v},\{\bar{m_j}\}_{j \in A_h}$ <br/> $A' = A S^{r}\pmod{n}$ <br/> $v' = v - e r$ <br/> $e' = e - \beta$  <br/> $ T = A'^{\bar{e}} S^{\bar{v}} \Bigg(\prod_{i \in A_h}{R_i^{\bar{m_i}}} \Bigg) \pmod{n} $ <br/> $D = H(T,n_1)$ <br/> $\hat{v} = \bar{v} + Dv'$ <br/> $\hat{e} = \bar{e} + De'$ <br/> $\{\hat{m_j} = \bar{m_j} + Dm_j\}_{j \in A_h}$ |
| 3. | $\hat{T} = \Bigg(\frac{Z}{A'^\beta \prod_{i \in A_r}{R_i^{m_i}}}\Bigg)^{-D} A'^{\hat{e}} S^{\hat{v}} \Bigg(\prod_{i \in A_h}{R_i^{\hat{m_i}}}\Bigg) \pmod{n} $ <br/> $\hat{D} = H(\hat{T},n_1)$ <br/><br/> Verify $\hat{D} == D$ |  |  |


#### Flow diagram with predicate

$\def \Prc {Pr_C=(A',\hat{e},\hat{v},\{\hat{m_j}\}_{j \in A_h})}$
$\def \Prp {Pr_p =( \{\hat{u_i}\}, \{\hat{r_i}\},\hat{r_{\Delta}},\hat{\alpha},\{\hat{m_j}\}_{j \in A_h})}$

| Step | Verifier | Public <br/> $n, S, Z, \{{R_i}\}_{i \in A},\{m_i\}_{i \in A_r}$ <br/> $\text{System parameter: } \beta$ <br/> $\text{Secure hash function: } H$  | User <br/> $\text{Secret: } A, e, v,\{m_i\}_{i \in A_h}$ |
| --- | --- | --- | --- |
| 0. | Random $n_1$ | $\xrightarrow[\hspace{4cm}]{\text{Proof request and } n_1}$ |  |
| 1. |  |  | Generate randomized signature $(A', e', v')$  <br/> Random $r$ <br/> $A' = A S^{r}\pmod{n}$ <br/> $v' = v - e r$ <br/> $e' = e - \beta$  <br/><br/> Generate credential proof <br/> Random $\bar{e},\bar{v},\{\bar{m_j}\}_{j \in A_h}$ <br/> $ T = A'^{\bar{e}} S^{\bar{v}} \Bigg(\prod_{i \in A_h}{R_i^{\bar{m_i}}} \Bigg) \pmod{n} $ |
| 2. |  |  | Generate randomized predicate proof result $(\{T_i\}_{1\leq i \leq 4},T_{\Delta})$ <br/> Random $r_1, r_2, r_3, r_4, r_{\Delta}$ <br/> $ \Delta = u_1^2 + u_2^2 +u_3^2 + u_4^2$ <br/><br/> $ \{T_i = Z^{u_i}S^{r_i}\pmod{n} \}_{1\leq i \leq 4}$ <br/> $ T_{\Delta} =  Z^{\Delta}S^{r_{\Delta}} \pmod{n} $ <br/><br/> $\mathcal{C}$ is now $(A',{T_1},{T_2},{T_3},{T_4},{T_{\Delta}})$|
| 3. |  |  | Generate predicate proof result $(\{\bar{T}_i\}_{1\leq i \leq 4},\bar{T}_{\Delta})$ <br/> Random $\bar{u_1},\bar{u_2},\bar{u_3},\bar{u_4}$ <br/> Random $\bar{r_1},\bar{r_2},\bar{r_3},\bar{r_4},\bar{r_{\Delta}}$ <br/><br/> $ \{\bar{T_i} = Z^{\bar{u_i}}S^{\bar{r_i}}\pmod{n} \}_{1\leq i \leq 4}$ <br/> $ \bar{T_{\Delta}} =  Z^{\bar{m_j}}(S^{b})^{\bar{r_{\Delta}}} \pmod{n} $ <br/><br/> Random $\bar{\alpha}$ <br/> $ \bar{K} = S^{\bar{\alpha}}T_1^{\bar{u_1}}T_2^{\bar{u_2}}T_3^{\bar{u_3}} T_4^{\bar{u_4}}\pmod{n}$ <br/><br/>$\mathcal{T}$ is now $(T,\bar{T_1},\bar{T_2},\bar{T_3},\bar{T_4},\bar{T_{\Delta}},\bar{K})$|
| 4. |  |  | Generate hash value <br/> $D = H(\mathcal{T},\mathcal{C},n_1)$  <br/><br/> Blind signature <br/> $\hat{v} = \bar{v} + Dv'$ <br/> $\hat{e} = \bar{e} + De'$ <br/> $\{\hat{m_j} = \bar{m_j} + Dm_j\}_{j \in A_h}$ |
| 5. |  |  | ${\alpha} = r_{\Delta}- u_1r_1 - u_2r_2 - u_3r_3 - u_4r_4 $ <br/><br/> Blind predicate <br/> $ \{\hat{u_i} = \bar{u_i}+Du_i \}_{1\leq i \leq 4}$ <br/> $ \{\hat{r_i} = \bar{r_i}+Dr_i \}_{1\leq i \leq 4}$ <br/> $\hat{r_{\Delta}} = \bar{r_{\Delta}}+Dr_{\Delta}$ <br/> $ \hat{\alpha} = \bar{\alpha}+D\alpha $ |
| 6. | | $\xleftarrow[\hspace{4cm}]{(D,\{Pr_C\},\{Pr_p\},\mathcal{C})}$ | The values $\Prc$ are the sub-proof for credential $C$. <br/><br/> The values $\Prp$ are the sub-proof for predicate $p$. <br/><br/> Then $(D,\{Pr_C\},\{Pr_p\},\mathcal{C})$ is the full proof sent to the Verifier.|
| 7. | $\hat{T} = \Bigg(\frac{Z}{A'^\beta \prod_{i \in A_r}{R_i^{m_i}}}\Bigg)^{-D} A'^{\hat{e}} S^{\hat{v}} \Bigg(\prod_{i \in A_h}{R_i^{\hat{m_i}}}\Bigg) \pmod{n} $ |  |  |
| 8. | $ \{\hat{T_i} = T_i^{-D}Z^{\hat{u_i}} S^{\hat{r_i}}\pmod{n} \}_{1\leq i \leq 4}$ <br/> $ \hat{T_{\Delta}} = \left((T_{\Delta})^bZ^{\Delta'}\right)^{-D}Z^{\hat{m_j}}(S^b)^{\hat{r_{\Delta}}}\pmod{n} $ <br/> $ \hat{K} = S^{\hat{\alpha}}(T_{\Delta})^{-D}T_1^{\hat{u_1}}T_2^{\hat{u_2}}T_3^{\hat{u_3}}T_4^{\hat{u_4}} \pmod{n} $ <br/><br/> $\hat{\mathcal{T}}$ is now $(\hat{T_1},\hat{T_2} ,\hat{T_3},\hat{T_4},\hat{T_{\Delta}},\hat{K}) $ |  |  |
| 9. | Generate hash value <br/> $\hat{D} = H(\mathcal{\hat{T}},\mathcal{C},n_1)$ <br/><br/> Verify $\hat{D} == D$ |  |  |



## Verifier sends a proof request
Along with a proof request the Verifier send nounce $n_1$ to User

In [8]:
nounce = number.getRandomNBitInteger(80)

## User generate proof

In [9]:
# public attributes
Rr = pow(R_4, m_4, n) * pow(R_5, m_5, n)
# private attributes
Rh = pow(R_1, m_1, n) * pow(R_2, m_2, n) * pow(R_3, m_3, n) * pow(R_6, m_6, n)

# Generate random value for each non-revealed attribute
m1_bar = number.getRandomNBitInteger(nsize)
m2_bar = number.getRandomNBitInteger(nsize)
m3_bar = number.getRandomNBitInteger(nsize)
m6_bar = number.getRandomNBitInteger(nsize)

# Calculate dot bar
Rh_bar = pow(R_1, m1_bar, n) * pow(R_2, m2_bar, n) * pow(R_3, m3_bar, n) * pow(R_6, m6_bar, n)

## User generate randomized signature $(A', e', v')$

In [10]:
# random r
r = number.getRandomNBitInteger(msize*nsize + 1)
# Calculate
A_prime = A * pow(S, r, n)
v_prime = v - e*r

# radnom beta
beta = number.getRandomNBitInteger(bitsize + 1)
# Calculate
e_prime = e - beta

User has generated the randomized signature $(A', e', v')$ and the original values $(A, e, v)$ are kept secret. 

### Why it works

In [11]:
# check the code is correct, (not part of the protocol)
Q_hat = pow(A_prime, e, n) * pow(S, -e*r, n) % n
assert(Q_hat == Q)

## User complete proof

In [12]:
# Generate
e_bar = number.getRandomNBitInteger(nsize + 1)
v_bar = number.getRandomNBitInteger(msize*nsize+1)
# Calculate
T = (pow(A_prime, e_bar, n) * pow(S, v_bar, n) * Rh_bar) % n

add $A'$ to $\mathcal{C}$ and $T$ to $\mathcal{T}$.


### Check predicates such as "A is older than 18 years old".

In [13]:
# select m6 to do the predicate
mp1 = m_6
mp1_bar = m6_bar

b = 1
ap1 = mp1 - 22

## User generate predicate proof

### Calculate $T$ equations

In [14]:
# Compute delta
delta = mp1 - ap1
roots = four_squares(delta)
# check delta is greater than > 0
assert(sum(roots) > 0)

# Generate random number
r1 = number.getRandomNBitInteger(msize*nsize + 1)
r2 = number.getRandomNBitInteger(msize*nsize + 1)
r3 = number.getRandomNBitInteger(msize*nsize + 1)
r4 = number.getRandomNBitInteger(msize*nsize + 1)
rd = number.getRandomNBitInteger(msize*nsize + 1)

# Compute T equations
T1 = pow(Z, roots[0], n) * pow(S, r1, n)
T2 = pow(Z, roots[1], n) * pow(S, r2, n)
T3 = pow(Z, roots[2], n) * pow(S, r3, n)
T4 = pow(Z, roots[3], n) * pow(S, r4, n)
Td = pow(Z, delta, n) * pow(S, rd, n)

and add these values to $\mathcal{C}$ in the order ${T_1},{T_2},{T_3},{T_4},{T_{\Delta}}$. 

$\mathcal{C}$ is now $(A',{T_1},{T_2},{T_3},{T_4},{T_{\Delta}})$


### Calculate $\bar{T}$ and $\bar{K}$ equations

In [15]:
# Generate random number
u1_bar = number.getRandomNBitInteger(nsize + 1)
u2_bar = number.getRandomNBitInteger(nsize + 1)
u3_bar = number.getRandomNBitInteger(nsize + 1)
u4_bar = number.getRandomNBitInteger(nsize + 1)

r1_bar = number.getRandomNBitInteger(nsize + 1)
r2_bar = number.getRandomNBitInteger(nsize + 1)
r3_bar = number.getRandomNBitInteger(nsize + 1)
r4_bar = number.getRandomNBitInteger(nsize + 1)
rd_bar = number.getRandomNBitInteger(nsize + 1)

# Compute T_bar equations
T1_bar = pow(Z, u1_bar, n) * pow(S, r1_bar, n) % n
T2_bar = pow(Z, u2_bar, n) * pow(S, r2_bar, n) % n
T3_bar = pow(Z, u3_bar, n) * pow(S, r3_bar, n) % n
T4_bar = pow(Z, u4_bar, n) * pow(S, r4_bar, n) % n
Td_bar = pow(Z, mp1_bar, n) * pow(S, b*rd_bar, n) % n

# Compute K_bar
alpha_bar = number.getRandomNBitInteger(msize*nsize + 1)
K_bar = pow(S, alpha_bar, n) * pow(T1, u1_bar, n) * pow(T2, u2_bar, n) * pow(T3, u3_bar, n) * pow(T4, u4_bar, n) % n


and add this values to $\mathcal{T}$ in the order $\bar{T_1},\bar{T_2},\bar{T_3},\bar{T_4},\bar{T_{\Delta}},\bar{K}$.

$\mathcal{T}$ is now $(T,\bar{T_1},\bar{T_2},\bar{T_3},\bar{T_4},\bar{T_{\Delta}},\bar{K})$

## Hashing

Compute the challenge
$$
D\leftarrow H(\mathcal{T},\mathcal{C},n_1)
$$

In [16]:
# build hash array
C = [A_prime, T1, T2, T3, T4, Td]
tau = [T, T1_bar, T2_bar, T3_bar, T4_bar, Td_bar, K_bar]
attr = str([tau, C, nounce])

# hashing
c_hash = hashlib.sha256(str(attr).encode('utf-8')).hexdigest()
# in code 'D' is 'c'
c = int(c_hash, base=32)

## User generate sub-proof $Pr_C$ for credential $C$.


In [17]:
# Compute
e_hat = e_bar + c*e_prime
v_hat = v_bar + c*v_prime

# For each none revealed attribute
m1_hat = m1_bar + c*m_1
m2_hat = m2_bar + c*m_2
m3_hat = m3_bar + c*m_3
m6_hat = m6_bar + c*m_6
# used for predicate
mp1_hat = m6_hat

# Calculate dot hat
Rh_hat = pow(R_1, m1_hat, n) * pow(R_2, m2_hat, n) * pow(R_3, m3_hat, n) * pow(R_6, m6_hat, n)

The values $Pr_C=(A',\hat{e},\hat{v},\{\hat{m_j}\}_{j \in A_h})$ are the sub-proof for credential $C$.


### Why it works

In [18]:
# check the code is correct, (not part of the protocol)
assert((pow(Rr, c, n) * Rh_hat % n) == (pow(Rr, c, n) * pow(Rh, c, n) * Rh_bar % n))

## User generate sub-proof $Pr_p$ for predicate $p$.


In [19]:
# For 1 ≤ i ≤ 4 compute
u1_hat = u1_bar + c*roots[0]
u2_hat = u2_bar + c*roots[1]
u3_hat = u3_bar + c*roots[2]
u4_hat = u4_bar + c*roots[3]

# For 1 ≤ i ≤ 4 compute
r1_hat = r1_bar + c*r1
r2_hat = r2_bar + c*r2
r3_hat = r3_bar + c*r3
r4_hat = r4_bar + c*r4

# Compute
rd_hat = rd_bar + c*rd

# Compute
alpha = rd - roots[0]*r1 - roots[1]*r2 - roots[2]*r3 - roots[3]*r4
alpha_hat = alpha_bar + c*alpha

The values $Pr_p =( \{\hat{u_i}\}, \{\hat{r_i}\},\hat{r_{\Delta}},\hat{\alpha},\{\hat{m_j}\}_{j \in A_h})$ are the sub-proof for predicate $p$.


### Why it works

In [20]:
# check the code is correct, (not part of the protocol)
assert(pow(T1, u1_hat, n) == pow(T1, u1_bar, n) * pow(T1, c*roots[0], n) % n)
assert(pow(T2, u2_hat, n) == pow(T2, u2_bar, n) * pow(T2, c*roots[1], n) % n)
assert(pow(T3, u3_hat, n) == pow(T3, u3_bar, n) * pow(T3, c*roots[2], n) % n)
assert(pow(T4, u4_hat, n) == pow(T4, u4_bar, n) * pow(T4, c*roots[3], n) % n)

## User sent full proof to the Verifier

The values $Pr_p =( \{\hat{u_i}\}, \{\hat{r_i}\},\hat{r_{\Delta}},\hat{\alpha},\{\hat{m_j}\}_{j \in A_h})$ are the sub-proof for predicate $p$.

The values $Pr_C=(A',\hat{e},\hat{v},\{\hat{m_j}\}_{j \in A_h})$ are the sub-proof for credential $C$.

Then $(D,\{Pr_C\},\{Pr_p\},\mathcal{C})$ is the full proof sent to the Verifier.

# Verification

## Verifier validate proof
Verifier generate sub-proof results $\hat{T}$ with the public key $(n, S, Z, \{{R_i}\}_{i \in A})$ and the received $(D, A',\hat{e},\hat{v},\{\hat{m_j}\}_{j \in A_h})$.

In [21]:
T_hat = (pow(Z, -c, n) * pow(A_prime, c*beta, n) * pow(A_prime, e_hat, n) * pow(S, v_hat, n) * pow(Rr, c, n) * Rh_hat) % n

and add $\hat{T}$ to $\hat{\mathcal{T}}$.


## Verifier validate predicate
Verifier using $(\{\hat{u_i}\}, \{\hat{r_i}\},\hat{r_{\Delta}},\hat{\alpha},\{\hat{m_j}\}_{j \in A_h})$ and calculate $\hat{T}$ and $\hat{K}$ equations for each predicate $p$.


In [22]:
delta_prim = ap1

# Compute T_hat equations
T1_hat = pow(T1, -c, n) * pow(Z, u1_hat, n) * pow(S, r1_hat, n) % n
T2_hat = pow(T2, -c, n) * pow(Z, u2_hat, n) * pow(S, r2_hat, n) % n
T3_hat = pow(T3, -c, n) * pow(Z, u3_hat, n) * pow(S, r3_hat, n) % n
T4_hat = pow(T4, -c, n) * pow(Z, u4_hat, n) * pow(S, r4_hat, n) % n
Td_hat = pow(Td, -b*c, n) * pow(Z, -delta_prim*c, n) *  pow(Z, mp1_hat, n) * pow(S, b * rd_hat, n)  % n


# Compute K_hat
K_hat = pow(S, alpha_hat, n) * pow(Td, -c, n) * pow(T1, u1_hat, n) * pow(T2, u2_hat, n) * pow(T3, u3_hat, n) * pow(T4, u4_hat, n) % n

and add these values to  $\hat{\mathcal{T}}$ in the order $\hat{T_1},\hat{T_2} ,\hat{T_3},\hat{T_4},\hat{T_{\Delta}},\hat{K}    $.

## Hashing

Compute the challenge
$$
\hat{D}\leftarrow H(\hat{\mathcal{T}},\mathcal{C},n_1)
$$

In [23]:
# build hash array
tau_hat = [T_hat, T1_hat, T2_hat, T3_hat, T4_hat, Td_hat, K_hat]
attr = str([tau_hat, C, nounce])

# hashing
c_hash_hat = hashlib.sha256(str(attr).encode('utf-8')).hexdigest()
# in code 'D' is 'c'
c_hat = int(c_hash_hat, base=32)

## Equality check
If $D=\hat{D}$ output VERIFIED else FAIL.

In [24]:
# Check equality
assert([T, c] == [T_hat, c_hat])

In [25]:
print('The equality has been VERIFIED when this line is printed!')

The equality has been VERIFIED when this line is printed!
